# 감성점수 산출
## SNS 엑셀 파일에서 총 긍정어, 부정어, 중립어의 개수를 구한다.

In [4]:
import pandas as pd

In [5]:
import konlpy

In [3]:
f=pd.read_excel("aron.xlsx",header=None)

#### *aron: 아로나민 골드 SNS 글이 각 행으로 이루어진 엑셀파일

In [6]:
import konlpy
from konlpy.tag import Kkma
k = Kkma()

In [5]:
sns_list=[]
for i in range(len(f)) :
    sns_list.append(k.pos(str(f.iloc[i,0]))) #형태소 분석한 각 행들을 리스트에 담음

In [6]:
import csv

f = open('polarity5.csv', mode='r', encoding = 'cp949')
reader = csv.reader(f)
d = {} 
for line in reader:
    value = line[1]  #리스트로 반환됨
    key = line[0]
    d.update({key:value}) #"형태소/형태소종류"가 키이고 감성점수가 값인 딕셔너리 만듦
f.close()

#### *polarity5: 수정된 감성사전으로 ngram:"형태소/형태소종류" 와 max.value: 감성점수(1, 0, -1) 로 이루어짐
    
      -특징: VA, XR이 주된 형태소 종류이고 NNG, VA이 소수 추가됨 

In [7]:
dic_list =[]
for key in d.keys():
        key2 = key.replace('*','').replace(';',' ').replace('/',' ').split()
        key3 = tuple(key2)
        dic_list.append(key3) #(형태소, 형태소종류) 형태의 튜플들을 한 리스트에 담음
       
dic2={}
for z in range(len(dic_list)):
    dic2.update({(dic_list[z]):list(d.values())[z]}) #키가 튜플형태이고 값은 그대로인 새로운 딕셔너리 만듦 

In [8]:
same_list=[]
for m in range(len(sns_list)):
    for n in range(len(dic_list)):
        if dic_list[n] in sns_list[m]:
            same_list.append(dic_list[n]) #딕셔너리 키들중 각 SNS 행에 포함된 키들 리스트에 담음
#SNS 한 문장 내 중복 표현은 한번만 반환

In [9]:
wordAll=same_list
wordDic={}
for w in wordAll:
    wordCnt=wordAll.count(w);
    wordDic[w]=wordCnt #키가 공통 형태소이고 값이 빈도수인 딕셔너리 만듦

In [15]:
ngram=[]
polarity=[]
count=[]
for key, value in wordDic.items():
    for key2, value2 in dic2.items():
        if key == key2 :
            ngram.append(key)
            polarity.append(int(value2))
            count.append(int(value))            
        else:
            pass
data={'ngram': ngram, 'polarity':polarity, 'count':count} #차례로 "(형태소/형태소종류)", 감성점수(1,0,-1), 빈도수 의미  
df=pd.DataFrame(data, columns=['ngram','polarity','count'])
df=df.sort_values('count',ascending=False)
df.head(20)

,ngram,polarity,count
2,"(피로, NNG)",-1,520
16,"(좋, VA)",1,372
23,"(같, VA)",0,242
28,"(추천, NNG)",1,209
11,"(회복, NNG)",1,189
6,"(건강, NNG)",0,187
17,"(없, VA)",0,177
3,"(피곤, NNG)",-1,173
8,"(많, VA)",0,116
4,"(부작용, NNG)",-1,68


In [16]:
df_pos=df[df['polarity']==1] # 긍정어만 포함하는 sub데이터프레임 생성
print(df_pos.head(20)) 
print(df_pos['count'].sum()) # 총 긍정점수 반환

         ngram  polarity  count
16     (좋, VA)         1    372
28   (추천, NNG)         1    209
11   (회복, NNG)         1    189
7     (저렴, XR)         1     66
21   (유명, NNG)         1     64
24   (도움, NNG)         1     54
35    (괜찮, VA)         1     48
102  (감사, NNG)         1     42
37   (사랑, NNG)         1     37
58   (행복, NNG)         1     19
111    (싸, VA)         1     18
9    (최고, NNG)         1     13
50    (즐겁, VA)         1     12
73    (고맙, VA)         1     12
93    (맛있, VA)         1     10
41    (편하, VA)         1     10
68    (확실, XR)         1     10
30    (새롭, VA)         1      9
15     (젊, VA)         1      9
112    (낫, VA)         1      8
1385


In [13]:
df_neu=df[df['polarity']==0] # 중립어만 포함하는 sub데이터프레임 생성
print(df_neu.head(10))
print(df_neu['count'].sum()) # 총 중립점수 반환

          ngram  polarity  count
23      (같, VA)         0    242
6     (건강, NNG)         0    187
17      (없, VA)         0    177
8       (많, VA)         0    116
34      (크, VA)         0     62
12     (다르, VA)         0     54
25  (다이어트, NNG)         0     35
10      (높, VA)         0     32
31      (작, VA)         0     26
52      (쉽, VA)         0     25
1061


In [17]:
df_neg=df[df['polarity']==-1] # 부정어만 포함하는 sub데이터프레임 생성
print(df_neg.head(20))
print(df_neg['count'].sum()) # 총 부정점수 반환

          ngram  polarity  count
2     (피로, NNG)        -1    520
3     (피곤, NNG)        -1    173
4    (부작용, NNG)        -1     68
40     (힘들, VA)        -1     59
5      (아프, VA)        -1     36
20     (병, NNG)        -1     33
48    (걱정, NNG)        -1     26
39     (비싸, VA)        -1     16
0       (늦, VA)        -1     14
43      (죽, VV)        -1     13
85      (싫, VA)        -1     12
90    (불안, NNG)        -1     10
26     (약하, VA)        -1      9
75     (무겁, VA)        -1      7
1       (잦, VA)        -1      6
22     (어렵, VA)        -1      6
72     (무섭, VA)        -1      5
101    (나쁘, VA)        -1      5
74     (맛없, VA)        -1      4
60     (심각, XR)        -1      4
1067
